In [1]:
import re
from collections import defaultdict
import json
import os
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [2]:
cwd = os.getcwd()
image_folder = os.path.join(cwd, 'images_hayao')

In [3]:
if not os.path.exists(image_folder):
    os.mkdir(image_folder)

In [4]:
def get_image_link(filtered):
    for i in filtered:
        result = i.find('p', attrs = {'class': ''}).text
        #we only want movies directed by Hayao Miyazaki
        director_match = 'Director:\nHayao Miyazaki'
        if re.search(director_match, result):
            poster_link = i.find('div', attrs = {'class': 'lister-item-image float-left'}).find('img')['loadlate']
            images.append(poster_link)
            sub_link = i.find('h3', attrs = {'class': 'lister-item-header'}).find('a').get('href')
            l = 'https://www.imdb.com{sublink}?ref_=adv_li_i'.format(sublink = sub_link)
            links.append(l)
            r = requests.get(l)
            d = r.content
            f = BeautifulSoup(d).find('div', attrs = {'id': 'main_bottom'})
            storyline = f.find('div', attrs = {'class': 'inline canwrap'}).select('span')[0].text
            stories.append(storyline)
            img_links = f.find('div', attrs = {'class': 'mediastrip'}).find_all('a')
            for s in img_links:
                try:
                    img = s.select('img')[0]['loadlate']
                    images.append(img)
                except:
                    continue
        else:
            continue
    
    

In [5]:
def get_next_page(data):
    sub_page = BeautifulSoup(data).find('div', attrs = {'class': 'desc'}).find_all('a')[-1].get('href')
    next_page = 'https://www.imdb.com{subpage}&ref_=adv_nxt'.format(subpage = sub_page)
    return next_page

In [6]:
link = 'https://www.imdb.com/search/title/?companies=co0048420&ref_=adv_prv'
images = []
links = []
pages = [link]
stories = []
resp = requests.get(link)
data = resp.content
filtered = BeautifulSoup(data).find_all('div', attrs = {'class': 'lister-item mode-advanced'})
get_image_link(filtered)
while len(images) <= 200:
    link = get_next_page(data)
    if link not in pages:
        pages.append(link)
        print(link)
        resp = requests.get(link)
        data = resp.content
        filtered = BeautifulSoup(data).find_all('div', attrs = {'class': 'lister-item mode-advanced'})
        get_image_link(filtered)
    else:
        break


https://www.imdb.com/search/title/?companies=co0048420&start=51&ref_=adv_nxt
https://www.imdb.com/search/title/?companies=co0048420&start=101&ref_=adv_nxt
https://www.imdb.com/search/title/?companies=co0048420&start=151&ref_=adv_nxt


In [7]:
len(images)

105

In [9]:
len(stories)

22

In [11]:
for i in range(len(stories)):
    with open('storylines_hayao.txt', 'a+') as fb:
        fb.write(stories[i])

In [275]:
for i in range(len(images)):
    store_as = image_folder+'/'+str(i)+'.jpg'
    img_data = requests.get(images[i]).content
    with open(store_as, 'wb') as fb:
        fb.write(img_data)